Michael Clawson: HW 2-
Let's first import a few things that we'll need, as well as read in and take a cursory look at our data:

In [2279]:
%matplotlib inline
from sklearn.model_selection import learning_curve
from sklearn.metrics import make_scorer
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib as mpl
import sklearn as skl
from sklearn.model_selection import train_test_split

In [2280]:
train_df = pd.read_csv("C:\\Users\\michael.clawson\\Desktop\\School\\Machine Learning\\HW 2\\train.csv",
                      header = 0)

In [2281]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Let's see what kind of data we have, and see what we'll need to fill.

In [2282]:
train_df.info(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

This dataframe below will allow us to fill our test categorical variables with the mode of the training data.

In [2283]:
categorical_df = train_df[[
 'MSSubClass',
 'MSZoning',
 'Street',
 'Alley',
 'Utilities',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'YearRemodAdd',
 'Exterior1st',
 'Exterior2nd',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'TotRmsAbvGrd',
 'GarageCars',
 'GarageQual',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'MoSold',
 'YrSold',
 'SaleType',
 'SaleCondition']].copy()

In [2284]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [2285]:
numeric_cols = [x for x in train_df.columns if ('Area' in x) | ('SF' in x)] + ['LotFrontage','MiscVal','EnclosedPorch','3SsnPorch','ScreenPorch','OverallQual','OverallCond','YearBuilt','SalePrice']

for col in numeric_cols:
    train_df[col] = train_df[col].astype(float)
numeric_cols

['LotArea',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'PoolArea',
 'LotFrontage',
 'MiscVal',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'SalePrice']

Let's fill the numeric columns with the mean of that column:

In [2286]:
def calc_mean_and_fill(df, column_name):     
    mean = df[column_name].mean()     
    df[column_name] = df[column_name].fillna(mean)
    
columns = ['LotArea',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'PoolArea',
 'LotFrontage',
 'MiscVal',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'OverallQual',
 'OverallCond',
 'YearBuilt'] 
for c in columns:     
    calc_mean_and_fill(train_df, c)

These next categorical variables will be filled with the mode.

In [2287]:
train_df = train_df.fillna(train_df['Utilities'].value_counts().index[0])

In [2288]:
train_df = train_df.fillna(train_df['Neighborhood'].value_counts().index[0])

In [2289]:
train_df = train_df.fillna(train_df['Condition1'].value_counts().index[0])

In [2290]:
train_df = train_df.fillna(train_df['Condition2'].value_counts().index[0])

In [2291]:
train_df = train_df.fillna(train_df['YearRemodAdd'].value_counts().index[0])

In [2292]:
train_df = train_df.fillna(train_df['Exterior1st'].value_counts().index[0])

In [2293]:
train_df = train_df.fillna(train_df['Exterior2nd'].value_counts().index[0])

In [2294]:
train_df = train_df.fillna(train_df['ExterQual'].value_counts().index[0])

In [2295]:
train_df = train_df.fillna(train_df['ExterCond'].value_counts().index[0])

In [2296]:
train_df = train_df.fillna(train_df['Foundation'].value_counts().index[0])

In [2297]:
train_df = train_df.fillna(train_df['BsmtQual'].value_counts().index[0])

In [2298]:
train_df = train_df.fillna(train_df['BsmtCond'].value_counts().index[0])

In [2299]:
train_df = train_df.fillna(train_df['BsmtExposure'].value_counts().index[0])

In [2300]:
train_df = train_df.fillna(train_df['FullBath'].value_counts().index[0])

In [2301]:
train_df = train_df.fillna(train_df['HalfBath'].value_counts().index[0])

In [2302]:
train_df = train_df.fillna(train_df['BedroomAbvGr'].value_counts().index[0])

In [2303]:
train_df = train_df.fillna(train_df['TotRmsAbvGrd'].value_counts().index[0])

In [2304]:
train_df = train_df.fillna(train_df['GarageCars'].value_counts().index[0])

In [2305]:
train_df = train_df.fillna(train_df['GarageQual'].value_counts().index[0])

In [2306]:
train_df = train_df.fillna(train_df['PoolQC'].value_counts().index[0])

In [2307]:
train_df = train_df.fillna(train_df['Fence'].value_counts().index[0])

In [2308]:
train_df = train_df.fillna(train_df['MiscFeature'].value_counts().index[0])

In [2309]:
train_df = train_df.fillna(train_df['MoSold'].value_counts().index[0])

In [2310]:
train_df = train_df.fillna(train_df['YrSold'].value_counts().index[0])

In [2311]:
train_df = train_df.fillna(train_df['SaleType'].value_counts().index[0])

In [2312]:
train_df = train_df.fillna(train_df['SaleCondition'].value_counts().index[0])

In [2313]:
train_df = train_df.fillna(train_df['MSZoning'].value_counts().index[0])

In [2314]:
train_df = train_df.fillna(train_df['MSSubClass'].value_counts().index[0])

In [2315]:
train_df = train_df.fillna(train_df['Street'].value_counts().index[0])

In [2316]:
train_df = train_df.fillna(train_df['Alley'].value_counts().index[0])

Classifying the categorical values as categorical:

In [2317]:
categorical_cols = [x for x in train_df.columns if x not in numeric_cols]

for col in categorical_cols:
    train_df[col] = train_df[col].astype('category')
categorical_cols

['Id',
 'MSSubClass',
 'MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'MoSold',
 'YrSold',
 'SaleType',
 'SaleCondition']

Let's get some dummy variables:

In [2318]:
small_df = train_df[['MSZoning']].copy()
small_df['MSZoning'] = small_df['MSZoning'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['MSZoning']
train_df = train_df.join(new_small_df)

In [2319]:
small_df = train_df[['MSSubClass']].copy()
small_df['MSSubClass'] = small_df['MSSubClass'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['MSSubClass']
train_df = train_df.join(new_small_df)

In [2320]:
small_df = train_df[['Street']].copy()
small_df['Street'] = small_df['Street'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['Street']
train_df = train_df.join(new_small_df)

In [2321]:
small_df = train_df[['Alley']].copy()
small_df['Alley'] = small_df['Alley'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['Alley']
train_df = train_df.join(new_small_df)

In [2322]:
small_df = train_df[['Utilities']].copy()
small_df['Utilities'] = small_df['Utilities'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['Utilities']
train_df = train_df.join(new_small_df)

In [2323]:
small_df = train_df[['Neighborhood']].copy()
small_df['Neighborhood'] = small_df['Neighborhood'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['Neighborhood']
train_df = train_df.join(new_small_df)

In [2324]:
small_df = train_df[['Condition1']].copy()
small_df['Condition1'] = small_df['Condition1'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['Condition1']
train_df = train_df.join(new_small_df)

In [2325]:
small_df = train_df[['Condition2']].copy()
small_df['Condition2'] = small_df['Condition2'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['Condition2']
train_df = train_df.join(new_small_df)

In [2326]:
small_df = train_df[['YearRemodAdd']].copy()
small_df['YearRemodAdd'] = small_df['YearRemodAdd'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['YearRemodAdd']
train_df = train_df.join(new_small_df)

In [2327]:
small_df = train_df[['Exterior1st']].copy()
small_df['Exterior1st'] = small_df['Exterior1st'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['Exterior1st']
train_df = train_df.join(new_small_df)

In [2328]:
small_df = train_df[['Exterior2nd']].copy()
small_df['Exterior2nd'] = small_df['Exterior2nd'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['Exterior2nd']
train_df = train_df.join(new_small_df)

In [2329]:
small_df = train_df[['ExterQual']].copy()
small_df['ExterQual'] = small_df['ExterQual'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['ExterQual']
train_df = train_df.join(new_small_df)

In [2330]:
small_df = train_df[['ExterCond']].copy()
small_df['ExterCond'] = small_df['ExterCond'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['ExterCond']
train_df = train_df.join(new_small_df)

In [2331]:
small_df = train_df[['Foundation']].copy()
small_df['Foundation'] = small_df['Foundation'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['Foundation']
train_df = train_df.join(new_small_df)

In [2332]:
small_df = train_df[['BsmtQual']].copy()
small_df['BsmtQual'] = small_df['BsmtQual'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['BsmtQual']
train_df = train_df.join(new_small_df)

In [2333]:
small_df = train_df[['BsmtCond']].copy()
small_df['BsmtCond'] = small_df['BsmtCond'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['BsmtCond']
train_df = train_df.join(new_small_df)

In [2334]:
small_df = train_df[['BsmtExposure']].copy()
small_df['BsmtExposure'] = small_df['BsmtExposure'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['BsmtExposure']
train_df = train_df.join(new_small_df)

In [2335]:
small_df = train_df[['FullBath']].copy()
small_df['FullBath'] = small_df['FullBath'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['FullBath']
train_df = train_df.join(new_small_df)

In [2336]:
small_df = train_df[['HalfBath']].copy()
small_df['HalfBath'] = small_df['HalfBath'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['HalfBath']
train_df = train_df.join(new_small_df)

In [2337]:
small_df = train_df[['BedroomAbvGr']].copy()
small_df['BedroomAbvGr'] = small_df['BedroomAbvGr'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['BedroomAbvGr']
train_df = train_df.join(new_small_df)

In [2338]:
small_df = train_df[['TotRmsAbvGrd']].copy()
small_df['TotRmsAbvGrd'] = small_df['TotRmsAbvGrd'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['TotRmsAbvGrd']
train_df = train_df.join(new_small_df)

In [2339]:
small_df = train_df[['GarageCars']].copy()
small_df['GarageCars'] = small_df['GarageCars'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['GarageCars']
train_df = train_df.join(new_small_df)

In [2340]:
small_df = train_df[['GarageQual']].copy()
small_df['GarageQual'] = small_df['GarageQual'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['GarageQual']
train_df = train_df.join(new_small_df)

In [2341]:
small_df = train_df[['PoolQC']].copy()
small_df['PoolQC'] = small_df['PoolQC'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['PoolQC']
train_df = train_df.join(new_small_df)

In [2342]:
small_df = train_df[['Fence']].copy()
small_df['Fence'] = small_df['Fence'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['Fence']
train_df = train_df.join(new_small_df)

In [2343]:
small_df = train_df[['MiscFeature']].copy()
small_df['MiscFeature'] = small_df['MiscFeature'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['MiscFeature']
train_df = train_df.join(new_small_df)

In [2344]:
small_df = train_df[['MoSold']].copy()
small_df['MoSold'] = small_df['MoSold'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['MoSold']
train_df = train_df.join(new_small_df)

In [2345]:
small_df = train_df[['YrSold']].copy()
small_df['YrSold'] = small_df['YrSold'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['YrSold']
train_df = train_df.join(new_small_df)

In [2346]:
small_df = train_df[['SaleType']].copy()
small_df['SaleType'] = small_df['SaleType'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['SaleType']
train_df = train_df.join(new_small_df)

In [2347]:
small_df = train_df[['SaleCondition']].copy()
small_df['SaleCondition'] = small_df['SaleCondition'].astype('category')
new_small_df = pd.get_dummies(small_df)
del train_df['SaleCondition']
train_df = train_df.join(new_small_df)

The following data is being deleted to avoid having more variable columns than observations, which will bias our data.

In [2348]:
del train_df['LotShape']
del train_df['LandContour']
del train_df['LotConfig']
del train_df['LandSlope']
del train_df['BldgType']
del train_df['HouseStyle']
del train_df['RoofStyle']
del train_df['RoofMatl']
del train_df['MasVnrType']
del train_df['BsmtFinType1']
del train_df['BsmtFinType2']
del train_df['Heating']
del train_df['HeatingQC']
del train_df['CentralAir']
del train_df['Electrical']
del train_df['BsmtFullBath']
del train_df['BsmtHalfBath']
del train_df['KitchenAbvGr']
del train_df['KitchenQual']
del train_df['Functional']
del train_df['Fireplaces']
del train_df['FireplaceQu']
del train_df['GarageType']
del train_df['GarageYrBlt']
del train_df['GarageFinish']
del train_df['GarageCond']
del train_df['PavedDrive']

These columns are being deleeted for simplicity's sake, as I was unaware it was important to combine the test and training data to get dummies, and it was too late to switch my code by the time I found out.

In [2349]:
del train_df['BedroomAbvGr_8']
del train_df['Condition2_RRAe']
del train_df['Condition2_RRAn']
del train_df['Condition2_RRNn']
del train_df['Exterior1st_ImStucc']
del train_df['Exterior1st_Stone']
del train_df['Exterior2nd_Other']
del train_df['GarageCars_0']
del train_df['GarageCars_1']
del train_df['GarageCars_2']
del train_df['GarageCars_3']
del train_df['GarageCars_4']
del train_df['GarageQual_Ex']
del train_df['MiscFeature_TenC']
del train_df['PoolQC_Fa']
del train_df['TotRmsAbvGrd_14']
del train_df['TotRmsAbvGrd_2']
del train_df['Utilities_NoSeWa']

In [2350]:
train_df.head(5)

,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,65.0,8450.0,7.0,5.0,2003.0,196.0,706.0,0.0,150.0,...,0,0,0,1,0,0,0,0,1,0
1,2,80.0,9600.0,6.0,8.0,1976.0,0.0,978.0,0.0,284.0,...,0,0,0,1,0,0,0,0,1,0
2,3,68.0,11250.0,7.0,5.0,2001.0,162.0,486.0,0.0,434.0,...,0,0,0,1,0,0,0,0,1,0
3,4,60.0,9550.0,7.0,5.0,1915.0,0.0,216.0,0.0,540.0,...,0,0,0,1,1,0,0,0,0,0
4,5,84.0,14260.0,8.0,5.0,2000.0,350.0,655.0,0.0,490.0,...,0,0,0,1,0,0,0,0,1,0


Let's delineate our training and testing subset in our training data:

In [2351]:
train_df['LogSalePrice'] = np.log(train_df['SalePrice'])

In [2352]:
del train_df['SalePrice']

In [2353]:
inputs = train_df.drop('LogSalePrice', axis = 1)

In [2354]:
outputs = train_df['LogSalePrice']

Now, we will cross-validate, scale, and then try out a couple of different models.

In [2355]:
inputs_train, inputs_test, outputs_train, outputs_test = train_test_split(
...     inputs, outputs, test_size=0.2, random_state=42)

In [2356]:
ss = StandardScaler()
inputs = ss.fit_transform(inputs)

In [2357]:
linear_regression_model = SGDRegressor(tol=.0001, eta0=.01)
linear_regression_model.fit(inputs_train, outputs_train)
train_predictions = linear_regression_model.predict(inputs_train)
test_predictions = linear_regression_model.predict(inputs_test)

train_mse = mean_squared_error(outputs_train, train_predictions)
test_mse = mean_squared_error(outputs_test, test_predictions)
print("Train MSE: {}".format(train_mse))
print("Test MSE: {}".format(test_mse))

Train MSE: 4.767443070115534e+32
Test MSE: 2.3649331377199925e+32


In [2358]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {"eta0": [ .001, .003, .01, .03, .1, .3, 1, 3]}

linear_regression_model = SGDRegressor(tol=.0001)

n_iter_search = 8
random_search = RandomizedSearchCV(linear_regression_model, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=3, scoring='neg_mean_squared_error')
random_search.fit(inputs_train, outputs_train)

print("Best Parameters: {}".format(random_search.best_params_))
print("Best Negative MSE: {}".format(random_search.best_score_))

Best Parameters: {'eta0': 0.001}
Best Negative MSE: -2.2101222946257158e+31


I'm not too wild about the MSE here, so let's try a different model:

In [2359]:
from sklearn.linear_model import ElasticNetCV
clf = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], alphas=[.01, .1, 1, 10, 100, 1000, 10000], max_iter=10000)
clf.fit(inputs_train, outputs_train)
train_predictions = clf.predict(inputs_train)
test_predictions = clf.predict(inputs_test)
print("Train MSE: {}".format(mean_squared_error(outputs_train, train_predictions)))
print("Test MSE: {}".format(mean_squared_error(outputs_test, test_predictions)))

Train MSE: 0.017354271291045484
Test MSE: 0.020132917816324394


The Elastic net wasn't too bad, but let's try another just for fun:

In [2361]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
reg = KNeighborsRegressor()
gridsearch = GridSearchCV(reg, {"n_neighbors": [1, 3, 5, 7, 9, 11], "weights": ['uniform', 'distance'], 
                                'p': [1, 2, 3]}, scoring='neg_mean_squared_error')
gridsearch.fit(inputs_train, outputs_train)
print("Best Params: {}".format(gridsearch.best_params_))
y_pred_train = gridsearch.predict(inputs_train)
y_pred_test = gridsearch.predict(inputs_test)
print("Train MSE: {}\tTest MSE: {}".format(mean_squared_error(outputs_train, y_pred_train),
                                           mean_squared_error(outputs_test, y_pred_test)))

Best Params: {'n_neighbors': 11, 'p': 1, 'weights': 'distance'}
Train MSE: 0.0	Test MSE: 0.04843453952789588


11 Nearest neighbors has the best fit, and further testing revealed that going higher did not improve the MSE. Let's just validate that these house prices look like house prices:

In [2363]:
real_predict = np.exp(y_pred_test)

In [2364]:
print(real_predict)

[ 138566.80671089  299239.25682249  121170.56388709  134548.86584424
  235914.48076636   91826.7558603   207061.78439542  152125.9822401
   92256.21500348  159841.85162017  129085.0971295   123884.42365739
  144317.44739837  190934.50764551  181149.00564512  117227.00946252
  182581.36068482  135437.90136135  106129.21952227  190363.05609036
  177048.0037749   174520.89473682  177552.50213293  110525.26028528
  176062.48671499  150269.762178    162646.8858636   146223.07927157
  192288.31052946  183465.7916728   118356.69310949  271612.14190667
  187874.4344541   119433.9781288   203890.81822682  140891.86631407
  156158.60718461  166266.13445982  287459.45768509  118702.11769646
  106839.16336883  212205.22097302  125127.94800366  281823.72505496
  143538.13185182  164991.16653895  119462.03602631  129759.85242826
  346672.15490822  156307.67496419  120542.52447792  210755.44313802
  135972.97178574  274318.5317145   138107.02961627  281351.76099481
  192162.28015248  168866.58189533 

Now that we're confident with our training set, let's apply the same stuff to our testing set:

In [2365]:
test_df = pd.read_csv("C:\\Users\\michael.clawson\\Desktop\\School\\Machine Learning\\HW 2\\test.csv",
                      header = 0)

Variable definition:

In [2366]:
numeric_cols = [x for x in test_df.columns if ('Area' in x) | ('SF' in x)] + ['LotFrontage','MiscVal','EnclosedPorch','3SsnPorch','ScreenPorch','OverallQual','OverallCond','YearBuilt']

for col in numeric_cols:
    test_df[col] = test_df[col].astype(float)
numeric_cols

['LotArea',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'PoolArea',
 'LotFrontage',
 'MiscVal',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'OverallQual',
 'OverallCond',
 'YearBuilt']

In [2367]:
def calc_mean_and_fill(df, column_name):     
    mean = train_df[column_name].mean()     
    df[column_name] = df[column_name].fillna(mean)
    
columns = ['LotArea',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'PoolArea',
 'LotFrontage',
 'MiscVal',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'OverallQual',
 'OverallCond',
 'YearBuilt'] 
for c in columns:     
    calc_mean_and_fill(test_df, c)

Filling categorical variables' NA's with the mode of the training set:

In [2368]:
test_df = test_df.fillna(categorical_df['Utilities'].value_counts().index[0])

In [2369]:
test_df = test_df.fillna(categorical_df['Neighborhood'].value_counts().index[0])

In [2370]:
test_df = test_df.fillna(categorical_df['Condition1'].value_counts().index[0])

In [2371]:
test_df = test_df.fillna(categorical_df['Condition2'].value_counts().index[0])

In [2372]:
test_df = test_df.fillna(categorical_df['YearRemodAdd'].value_counts().index[0])

In [2373]:
test_df = test_df.fillna(categorical_df['Exterior1st'].value_counts().index[0])

In [2374]:
test_df = test_df.fillna(categorical_df['Exterior2nd'].value_counts().index[0])

In [2375]:
test_df = test_df.fillna(categorical_df['ExterQual'].value_counts().index[0])

In [2376]:
test_df = test_df.fillna(categorical_df['ExterCond'].value_counts().index[0])

In [2377]:
test_df = test_df.fillna(categorical_df['Foundation'].value_counts().index[0])

In [2378]:
test_df = test_df.fillna(categorical_df['BsmtQual'].value_counts().index[0])

In [2379]:
test_df = test_df.fillna(categorical_df['BsmtCond'].value_counts().index[0])

In [2380]:
test_df = test_df.fillna(categorical_df['BsmtExposure'].value_counts().index[0])

In [2381]:
test_df = test_df.fillna(categorical_df['FullBath'].value_counts().index[0])

In [2382]:
test_df = test_df.fillna(categorical_df['HalfBath'].value_counts().index[0])

In [2383]:
test_df = test_df.fillna(categorical_df['BedroomAbvGr'].value_counts().index[0])

In [2384]:
test_df = test_df.fillna(categorical_df['TotRmsAbvGrd'].value_counts().index[0])

In [2385]:
test_df = test_df.fillna(categorical_df['GarageCars'].value_counts().index[0])

In [2386]:
test_df = test_df.fillna(categorical_df['GarageQual'].value_counts().index[0])

In [2387]:
test_df = test_df.fillna(categorical_df['PoolQC'].value_counts().index[0])

In [2388]:
test_df = test_df.fillna(categorical_df['Fence'].value_counts().index[0])

In [2389]:
test_df = test_df.fillna(categorical_df['MiscFeature'].value_counts().index[0])

In [2390]:
test_df = test_df.fillna(categorical_df['MoSold'].value_counts().index[0])

In [2391]:
test_df = test_df.fillna(categorical_df['YrSold'].value_counts().index[0])

In [2392]:
test_df = test_df.fillna(categorical_df['SaleType'].value_counts().index[0])

In [2393]:
test_df = test_df.fillna(categorical_df['SaleCondition'].value_counts().index[0])

In [2394]:
test_df = test_df.fillna(categorical_df['MSZoning'].value_counts().index[0])

In [2395]:
test_df = test_df.fillna(categorical_df['MSSubClass'].value_counts().index[0])

In [2396]:
test_df = test_df.fillna(categorical_df['Street'].value_counts().index[0])

In [2397]:
test_df = test_df.fillna(categorical_df['Alley'].value_counts().index[0])

Variable definition:

In [2398]:
categorical_cols = [x for x in test_df.columns if x not in numeric_cols]

for col in categorical_cols:
    test_df[col] = test_df[col].astype('category')
categorical_cols

['Id',
 'MSSubClass',
 'MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'MoSold',
 'YrSold',
 'SaleType',
 'SaleCondition']

Getting some dummies:

In [2399]:
small_df = test_df[['MSZoning']].copy()
small_df['MSZoning'] = small_df['MSZoning'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['MSZoning']
test_df = test_df.join(new_small_df)

In [2400]:
small_df = test_df[['MSSubClass']].copy()
small_df['MSSubClass'] = small_df['MSSubClass'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['MSSubClass']
test_df = test_df.join(new_small_df)

In [2401]:
small_df = test_df[['Street']].copy()
small_df['Street'] = small_df['Street'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['Street']
test_df = test_df.join(new_small_df)

In [2402]:
small_df = test_df[['Alley']].copy()
small_df['Alley'] = small_df['Alley'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['Alley']
test_df = test_df.join(new_small_df)

In [2403]:
small_df = test_df[['Utilities']].copy()
small_df['Utilities'] = small_df['Utilities'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['Utilities']
test_df = test_df.join(new_small_df)

In [2404]:
small_df = test_df[['Neighborhood']].copy()
small_df['Neighborhood'] = small_df['Neighborhood'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['Neighborhood']
test_df = test_df.join(new_small_df)

In [2405]:
small_df = test_df[['Condition1']].copy()
small_df['Condition1'] = small_df['Condition1'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['Condition1']
test_df = test_df.join(new_small_df)

In [2406]:
small_df = test_df[['Condition2']].copy()
small_df['Condition2'] = small_df['Condition2'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['Condition2']
test_df = test_df.join(new_small_df)

In [2407]:
small_df = test_df[['YearRemodAdd']].copy()
small_df['YearRemodAdd'] = small_df['YearRemodAdd'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['YearRemodAdd']
test_df = test_df.join(new_small_df)

In [2408]:
small_df = test_df[['Exterior1st']].copy()
small_df['Exterior1st'] = small_df['Exterior1st'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['Exterior1st']
test_df = test_df.join(new_small_df)

In [2409]:
small_df = test_df[['Exterior2nd']].copy()
small_df['Exterior2nd'] = small_df['Exterior2nd'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['Exterior2nd']
test_df = test_df.join(new_small_df)

In [2410]:
small_df = test_df[['ExterQual']].copy()
small_df['ExterQual'] = small_df['ExterQual'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['ExterQual']
test_df = test_df.join(new_small_df)

In [2411]:
small_df = test_df[['ExterCond']].copy()
small_df['ExterCond'] = small_df['ExterCond'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['ExterCond']
test_df = test_df.join(new_small_df)

In [2412]:
small_df = test_df[['Foundation']].copy()
small_df['Foundation'] = small_df['Foundation'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['Foundation']
test_df = test_df.join(new_small_df)

In [2413]:
small_df = test_df[['BsmtQual']].copy()
small_df['BsmtQual'] = small_df['BsmtQual'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['BsmtQual']
test_df = test_df.join(new_small_df)

In [2414]:
small_df = test_df[['BsmtCond']].copy()
small_df['BsmtCond'] = small_df['BsmtCond'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['BsmtCond']
test_df = test_df.join(new_small_df)

In [2415]:
small_df = test_df[['BsmtExposure']].copy()
small_df['BsmtExposure'] = small_df['BsmtExposure'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['BsmtExposure']
test_df = test_df.join(new_small_df)

In [2416]:
small_df = test_df[['FullBath']].copy()
small_df['FullBath'] = small_df['FullBath'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['FullBath']
test_df = test_df.join(new_small_df)

In [2417]:
small_df = test_df[['HalfBath']].copy()
small_df['HalfBath'] = small_df['HalfBath'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['HalfBath']
test_df = test_df.join(new_small_df)

In [2418]:
small_df = test_df[['BedroomAbvGr']].copy()
small_df['BedroomAbvGr'] = small_df['BedroomAbvGr'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['BedroomAbvGr']
test_df = test_df.join(new_small_df)

In [2419]:
small_df = test_df[['TotRmsAbvGrd']].copy()
small_df['TotRmsAbvGrd'] = small_df['TotRmsAbvGrd'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['TotRmsAbvGrd']
test_df = test_df.join(new_small_df)

In [2420]:
small_df = test_df[['GarageCars']].copy()
small_df['GarageCars'] = small_df['GarageCars'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['GarageCars']
test_df = test_df.join(new_small_df)

In [2421]:
small_df = test_df[['GarageQual']].copy()
small_df['GarageQual'] = small_df['GarageQual'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['GarageQual']
test_df = test_df.join(new_small_df)

In [2422]:
small_df = test_df[['PoolQC']].copy()
small_df['PoolQC'] = small_df['PoolQC'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['PoolQC']
test_df = test_df.join(new_small_df)

In [2423]:
small_df = test_df[['Fence']].copy()
small_df['Fence'] = small_df['Fence'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['Fence']
test_df = test_df.join(new_small_df)

In [2424]:
small_df = test_df[['MiscFeature']].copy()
small_df['MiscFeature'] = small_df['MiscFeature'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['MiscFeature']
test_df = test_df.join(new_small_df)

In [2425]:
small_df = test_df[['MoSold']].copy()
small_df['MoSold'] = small_df['MoSold'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['MoSold']
test_df = test_df.join(new_small_df)

In [2426]:
small_df = test_df[['YrSold']].copy()
small_df['YrSold'] = small_df['YrSold'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['YrSold']
test_df = test_df.join(new_small_df)

In [2427]:
small_df = test_df[['SaleType']].copy()
small_df['SaleType'] = small_df['SaleType'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['SaleType']
test_df = test_df.join(new_small_df)

In [2428]:
small_df = test_df[['SaleCondition']].copy()
small_df['SaleCondition'] = small_df['SaleCondition'].astype('category')
new_small_df = pd.get_dummies(small_df)
del test_df['SaleCondition']
test_df = test_df.join(new_small_df)

Deletion of variables that don't seem to contribute much to housing prices:

In [2429]:
del test_df['LotShape']
del test_df['LandContour']
del test_df['LotConfig']
del test_df['LandSlope']
del test_df['BldgType']
del test_df['HouseStyle']
del test_df['RoofStyle']
del test_df['RoofMatl']
del test_df['MasVnrType']
del test_df['BsmtFinType1']
del test_df['BsmtFinType2']
del test_df['Heating']
del test_df['HeatingQC']
del test_df['CentralAir']
del test_df['Electrical']
del test_df['BsmtFullBath']
del test_df['BsmtHalfBath']
del test_df['KitchenAbvGr']
del test_df['KitchenQual']
del test_df['Functional']
del test_df['Fireplaces']
del test_df['FireplaceQu']
del test_df['GarageType']
del test_df['GarageYrBlt']
del test_df['GarageFinish']
del test_df['GarageCond']
del test_df['PavedDrive']

Variables I have to delete in order to get predictions to work at all:

In [2430]:
del test_df['SaleType_AllPub']
del test_df['MSZoning_AllPub']
del test_df['MSSubClass_150']
del test_df['Exterior2nd_AllPub']
del test_df['TotRmsAbvGrd_13']
del test_df['TotRmsAbvGrd_15']
del test_df['GarageCars_0.0']
del test_df['GarageCars_1.0']
del test_df['GarageCars_2.0']
del test_df['GarageCars_3.0']
del test_df['GarageCars_4.0']
del test_df['GarageCars_5.0']
del test_df['FullBath_4']
del test_df['GarageCars_AllPub']
del test_df['Exterior1st_AllPub']

Predictions:

In [2431]:
real_inputs_test = test_df

In [2432]:
real_pred_test = gridsearch.predict(real_inputs_test)


Verifying that my training and testing data have the same dimensions:

In [2433]:
set(train_df.columns).difference(set(real_inputs_test.columns))

{'LogSalePrice'}

In [2434]:
real_inputs_test.shape

(1459, 280)

In [2435]:
train_df.shape

(1460, 281)

In [2436]:
train_df.sort_index(axis=1).head(5)

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley_AllPub,Alley_Grvl,Alley_Pave,BedroomAbvGr_0,BedroomAbvGr_1,BedroomAbvGr_2,BedroomAbvGr_3,...,YearRemodAdd_2006,YearRemodAdd_2007,YearRemodAdd_2008,YearRemodAdd_2009,YearRemodAdd_2010,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010
0,856.0,854.0,0.0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,1262.0,0.0,0.0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,920.0,866.0,0.0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,961.0,756.0,0.0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,1145.0,1053.0,0.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [2437]:
test_df.sort_index(axis = 1).head(5)

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley_AllPub,Alley_Grvl,Alley_Pave,BedroomAbvGr_0,BedroomAbvGr_1,BedroomAbvGr_2,BedroomAbvGr_3,...,YearRemodAdd_2006,YearRemodAdd_2007,YearRemodAdd_2008,YearRemodAdd_2009,YearRemodAdd_2010,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010
0,896.0,0.0,0.0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,1329.0,0.0,0.0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,928.0,701.0,0.0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,926.0,678.0,0.0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,1280.0,0.0,0.0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [2438]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Columns: 280 entries, Id to SaleCondition_Partial
dtypes: category(1), float64(22), uint8(257)
memory usage: 671.3 KB


In [2439]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 281 entries, Id to LogSalePrice
dtypes: category(1), float64(23), uint8(257)
memory usage: 683.1 KB


In [2440]:
list(test_df.columns)

['Id',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MSZoning_C (all)',
 'MSZoning_FV',
 'MSZoning_RH',
 'MSZoning_RL',
 'MSZoning_RM',
 'MSSubClass_20',
 'MSSubClass_30',
 'MSSubClass_40',
 'MSSubClass_45',
 'MSSubClass_50',
 'MSSubClass_60',
 'MSSubClass_70',
 'MSSubClass_75',
 'MSSubClass_80',
 'MSSubClass_85',
 'MSSubClass_90',
 'MSSubClass_120',
 'MSSubClass_160',
 'MSSubClass_180',
 'MSSubClass_190',
 'Street_Grvl',
 'Street_Pave',
 'Alley_AllPub',
 'Alley_Grvl',
 'Alley_Pave',
 'Utilities_AllPub',
 'Neighborhood_Blmngtn',
 'Neighborhood_Blueste',
 'Neighborhood_BrDale',
 'Neighborhood_BrkSide',
 'Neighborhood_ClearCr',
 'Neighborhood_CollgCr',
 'Neighborhood_Crawfor',
 'Neighborhood_Edwards',
 'Neighbor

In [2441]:
list(train_df.columns)

['Id',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MSZoning_C (all)',
 'MSZoning_FV',
 'MSZoning_RH',
 'MSZoning_RL',
 'MSZoning_RM',
 'MSSubClass_20',
 'MSSubClass_30',
 'MSSubClass_40',
 'MSSubClass_45',
 'MSSubClass_50',
 'MSSubClass_60',
 'MSSubClass_70',
 'MSSubClass_75',
 'MSSubClass_80',
 'MSSubClass_85',
 'MSSubClass_90',
 'MSSubClass_120',
 'MSSubClass_160',
 'MSSubClass_180',
 'MSSubClass_190',
 'Street_Grvl',
 'Street_Pave',
 'Alley_AllPub',
 'Alley_Grvl',
 'Alley_Pave',
 'Utilities_AllPub',
 'Neighborhood_Blmngtn',
 'Neighborhood_Blueste',
 'Neighborhood_BrDale',
 'Neighborhood_BrkSide',
 'Neighborhood_ClearCr',
 'Neighborhood_CollgCr',
 'Neighborhood_Crawfor',
 'Neighborhood_Edwards',
 'Neighbor

In [2444]:
print(real_pred_test)

[ 11.85915089  12.1157252   12.07005964 ...,  12.17998507  11.77002168
  12.35497709]


Modifying the data to look like housing prices:

In [2445]:
test_real_predict = np.exp(real_pred_test)

In [2447]:
list(test_real_predict)

[141372.12749707908,
 182722.74018310936,
 174566.25557318385,
 178713.25391310459,
 164475.58033615447,
 162842.53299700981,
 145468.65427358414,
 175818.05832628891,
 162835.70434487949,
 127660.23605313584,
 219944.83098558336,
 93797.164941559793,
 99753.238826245113,
 152624.88113582911,
 106336.58090414846,
 257059.57298377575,
 197161.77913539603,
 204884.3503346637,
 264379.62474859913,
 362195.60496271623,
 246836.70636695213,
 188287.28449198563,
 153744.1448502696,
 177893.91165554911,
 162619.69235453277,
 179361.60690307719,
 215999.10110374712,
 207543.36726065804,
 182772.68365472779,
 188971.46900781366,
 184100.58413122696,
 125290.97748098709,
 185559.70675879758,
 217894.68874763828,
 275149.97100654797,
 252865.3685164754,
 140262.09166295119,
 152776.56658094903,
 137174.19112900703,
 136681.43620763122,
 160730.20096722181,
 124893.52290246956,
 227472.08364702965,
 176129.61369809913,
 204538.62111373787,
 215251.35281541449,
 257001.56481654916,
 203407.53007085

In [2450]:
len(test_real_predict)

1459

Send it to a csv to submit! I got like 3700th!

In [ ]:
import  csv

with open("predictions.csv","w") as f:
    wr = csv.writer(f,delimiter="\n")
    wr.writerow(test_real_predict)

While I initially deleted many variables to ensure that I did not have more variables than observations, I would definitely re-run the this analysis with all of the variables included, or run a model to tell me which ones didn't really matter. While I thought that I included the most pertinent categorical variables in predicting housing prices, that may not necessarily be the case. I personally felt like I did well given how much I knew, but I believe that Zillow might be better off paying someone who's coded for longer than 2 months to validate my findings rather than just taking mine at face value.